## Hungarian algorithm for MAXIMIZATION

In [1]:
from typing import List, Tuple

def hungarian_max(C: List[List[float]]) -> Tuple[float, List[Tuple[int, int]]]:
    """
    Hungarian algorithm for MAXIMIZATION.

    Returns:
      - total_cost (maximum total weight)
      - pairs as 1-based indices: [(i, j), ...] meaning i in {1..n} matched to j in {1..n}
    """
    n = len(C)
    if n == 0 or any(len(row) != n for row in C):
        raise ValueError("W must be a non-empty square n x n matrix.")

    # Dual potentials
    u = [max(row) for row in C]  # u_i = max_j w_{i,j}
    v = [0.0] * n                # v_j = 0

    # Matching: matchL[i] = j, matchR[j] = i, -1 means unmatched
    matchL = [-1] * n
    matchR = [-1] * n

    EPS = 1e-12

    def build_eq_adj() -> List[List[int]]:
        """Adjacency (equality subgraph) from each left i to rights j where u_i+v_j == w_ij."""
        adj = [[] for _ in range(n)]
        for i in range(n):
            ui = u[i]
            for j in range(n):
                if abs((ui + v[j]) - C[i][j]) <= EPS:
                    adj[i].append(j)
        return adj

    def try_augment(i: int, adj: List[List[int]], seenR: List[bool]) -> bool:
        """Kuhn DFS augmenting path in equality graph."""
        for j in adj[i]:
            if seenR[j]:
                continue
            seenR[j] = True
            if matchR[j] == -1 or try_augment(matchR[j], adj, seenR):
                matchL[i] = j
                matchR[j] = i
                return True
        return False

    def max_matching_in_eq() -> int:
        """Augment current matching to a maximum matching in current equality graph."""
        adj = build_eq_adj()
        for i in range(n):
            if matchL[i] == -1:
                seenR = [False] * n
                try_augment(i, adj, seenR)
        return sum(1 for i in range(n) if matchL[i] != -1)

    def min_vertex_cover_in_eq() -> Tuple[List[bool], List[bool]]:
        """
        Minimum vertex cover in bipartite equality graph from current maximum matching.
        Returns boolean arrays: in_cover_L, in_cover_R.
        Construction:
          Z = vertices reachable from unmatched left via alternating paths
          min cover = (L \\ Z) ∪ (R ∩ Z)
        """
        adj = build_eq_adj()

        visL = [False] * n
        visR = [False] * n

        from collections import deque
        q = deque()

        # start from unmatched left vertices
        for i in range(n):
            if matchL[i] == -1:
                visL[i] = True
                q.append(("L", i))

        while q:
            side, x = q.popleft()
            if side == "L":
                i = x
                for j in adj[i]:
                    # along NON-matching edges
                    if matchL[i] == j:
                        continue
                    if not visR[j]:
                        visR[j] = True
                        q.append(("R", j))
            else:
                j = x
                i2 = matchR[j]
                # along MATCHING edge back to left (if exists)
                if i2 != -1 and not visL[i2]:
                    visL[i2] = True
                    q.append(("L", i2))

        in_cover_L = [not visL[i] for i in range(n)]  # L \ Z
        in_cover_R = [visR[j] for j in range(n)]      # R ∩ Z
        return in_cover_L, in_cover_R

    # Initial matching in equality graph
    msize = max_matching_in_eq()

    while msize < n:
        in_cover_L, in_cover_R = min_vertex_cover_in_eq()

        # R = I ∩ Q  => left in cover
        # T = J ∩ Q  => right in cover
        left_not_in_cover = [i for i in range(n) if not in_cover_L[i]]   # I \ R
        right_not_in_cover = [j for j in range(n) if not in_cover_R[j]]  # J \ T
        right_in_cover = [j for j in range(n) if in_cover_R[j]]          # T

        # epsilon = min slack over (i in I\R, j in J\T)
        eps = float("inf")
        for i in left_not_in_cover:
            ui = u[i]
            for j in right_not_in_cover:
                slack = ui + v[j] - C[i][j]
                if slack < eps:
                    eps = slack

        if eps == float("inf"):
            raise RuntimeError("Could not compute epsilon; unexpected state.")
        if eps < 0:
            # clamp tiny negatives due to floating precision
            if eps > -1e-9:
                eps = 0.0
            else:
                raise RuntimeError("Negative epsilon encountered; check inputs/precision.")

        # Update potentials
        for i in left_not_in_cover:
            u[i] -= eps
        for j in right_in_cover:
            v[j] += eps

        # Augment to maximum matching again
        msize = max_matching_in_eq()

    # Build result
    pairs = [(i + 1, matchL[i] + 1) for i in range(n)]
    total_cost = sum(C[i][matchL[i]] for i in range(n))
    return total_cost, pairs

In [3]:
if __name__ == "__main__":
    C = [
        [20, 40, 10, 50],
        [100, 80, 30, 40],
        [10, 5, 60, 20],
        [70, 30, 10, 25]
    ]
    total, pairs = hungarian_max(C)
    print("Total cost:", total)
    print("Pairs (i, j):", pairs)

Total cost: 260
Pairs (i, j): [(1, 4), (2, 2), (3, 3), (4, 1)]


## Hungarian algorithm for MINIMALIZATION

#### The replacement involves subtracting each cost from the same constant, which makes minimizing the sum of costs equivalent to maximizing the sum of the transformed weights, because each match corresponds to a value that differs from the constant by exactly the sum of the costs.

In [4]:
from typing import List, Tuple

def hungarian_min(C: List[List[float]]) -> Tuple[float, List[Tuple[int, int]]]:
    """
    Uses hungarian_max by transforming a MIN cost matrix C into a MAX weight matrix W'.
    Returns:
      - min_total_cost
      - pairs (i, j) as 1-based indices
    """
    n = len(C)
    if n == 0 or any(len(row) != n for row in C):
        raise ValueError("C must be a non-empty square n x n matrix.")

    K = max(max(row) for row in C)  # >= all costs
    Wprime = [[K - C[i][j] for j in range(n)] for i in range(n)]

    max_total, pairs = hungarian_max(Wprime)  # <-- calls your max version

    # Convert back to minimal cost using the same chosen pairs
    min_total = 0.0
    for (i1, j1) in pairs:  # 1-based
        min_total += C[i1 - 1][j1 - 1]

    return min_total, pairs

In [6]:
if __name__ == "__main__":
    C = [
        [20, 40, 10, 50],
        [100, 80, 30, 40],
        [10, 5, 60, 20],
        [70, 30, 10, 25]
    ]
    min_cost, pairs = hungarian_min(C)
    print("Min cost:", min_cost)
    print("Pairs:", pairs)

Min cost: 75.0
Pairs: [(1, 1), (2, 4), (3, 2), (4, 3)]
